In [1]:
from typing import Annotated

from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import BaseMessage
from typing_extensions import TypedDict

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition


In [2]:
class State(TypedDict):
    messages: Annotated[list, add_messages]

graph_builder = StateGraph(State)

In [3]:
tool = TavilySearchResults(max_results=2)
tools = [tool]
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
llm_with_tools = llm.bind_tools(tools)


In [4]:
def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

graph_builder.add_node("chatbot", chatbot)

In [5]:
tool_node = ToolNode(tools=[tool])
graph_builder.add_node("tools", tool_node)

In [7]:
graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")

In [9]:
memory = MemorySaver()
graph = graph_builder.compile(checkpointer=memory)

In [10]:
config = {"configurable": {"thread_id": "1"}}
# 챗봇 채팅 yield 객체 생성
events = graph.stream(
    {
        "messages": [
            {
                "role": "user",
                "content": (
                    "I'm learning LangGraph"
                    "Could you do some research on it for me?"
                ),
            },
        ],
    },
    config,
    stream_mode="values",
)
# 챗봇 채팅 출력
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================ Human Message =================================

I'm learning LangGraphCould you do some research on it for me?
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (call_4Lm34EHkcpyMEyCAydgAoKFG)
 Call ID: call_4Lm34EHkcpyMEyCAydgAoKFG
  Args:
    query: LangGraph
================================= Tool Message =================================
Name: tavily_search_results_json

[{"title": "LangGraph Quickstart - GitHub Pages", "url": "https://langchain-ai.github.io/langgraph/tutorials/introduction/", "content": "[](https://langchain-ai.github.io/langgraph/tutorials/introduction/#__codelineno-9-1)Assistant: LangGraph is a library designed to help build stateful multi-agent applications using language models. It provides tools for creating workflows and state machines to coordinate multiple AI agents or language model interactions. LangGraph is built on top of LangChain, leveraging its c

In [11]:
events = graph.stream(
    {
        "messages": [
            {
                "role": "user",
                "content": (
                    "Ya that's helpful. Maybe I'll "
                    "build an autonomous agent with it!"
                ),
            },
        ],
    },
    config,
    stream_mode="values",
)
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================ Human Message =================================

Ya that's helpful. Maybe I'll build an autonomous agent with it!
================================== Ai Message ==================================

That sounds like an exciting project! Building an autonomous agent with LangGraph can open up many possibilities for creating intelligent applications. If you have any specific ideas or need further assistance as you work on your project, feel free to ask. Good luck with your development!


In [12]:
to_replay = None
for state in graph.get_state_history(config):
    print("Num messages: ", len(state.values["messages"]), "Next: ", state.next)
    print("-" * 80)
    # 메시지가 6개인 상태를 찾아서 저장
    if len(state.values["messages"]) == 6:
        to_replay = state

Num messages:  6 Next:  ()
--------------------------------------------------------------------------------
Num messages:  5 Next:  ('chatbot',)
--------------------------------------------------------------------------------
Num messages:  4 Next:  ('__start__',)
--------------------------------------------------------------------------------
Num messages:  4 Next:  ()
--------------------------------------------------------------------------------
Num messages:  3 Next:  ('chatbot',)
--------------------------------------------------------------------------------
Num messages:  2 Next:  ('tools',)
--------------------------------------------------------------------------------
Num messages:  1 Next:  ('chatbot',)
--------------------------------------------------------------------------------
Num messages:  0 Next:  ('__start__',)
--------------------------------------------------------------------------------


In [13]:
print(to_replay.next)
print(to_replay.config)

()
{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f00a0fb-95fd-6486-8006-b2b0bd3a98a0'}}


In [14]:
for event in graph.stream(None, to_replay.config, stream_mode="values"):
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================== Ai Message ==================================

That sounds like an exciting project! Building an autonomous agent with LangGraph can open up many possibilities for creating intelligent applications. If you have any specific ideas or need further assistance as you work on your project, feel free to ask. Good luck with your development!
